In [1]:
import librosa
from pydub import AudioSegment
from gtts import gTTS
from tempfile import TemporaryFile
import sys

In [3]:
sys.path.remove("/home/kian/.ipython")

In [4]:
sys.path.append("/usr/bin/ffmpeg")

['',
 '/home/kian/ML/SBHacks',
 '/home/kian/ML/SBHacks/sbhacks/lib/python2.7',
 '/home/kian/ML/SBHacks/sbhacks/lib/python2.7/plat-x86_64-linux-gnu',
 '/home/kian/ML/SBHacks/sbhacks/lib/python2.7/lib-tk',
 '/home/kian/ML/SBHacks/sbhacks/lib/python2.7/lib-old',
 '/home/kian/ML/SBHacks/sbhacks/lib/python2.7/lib-dynload',
 '/usr/lib/python2.7',
 '/usr/lib/python2.7/plat-x86_64-linux-gnu',
 '/usr/lib/python2.7/lib-tk',
 '/home/kian/ML/SBHacks/sbhacks/local/lib/python2.7/site-packages',
 '/home/kian/ML/SBHacks/sbhacks/lib/python2.7/site-packages',
 '/home/kian/ML/SBHacks/sbhacks/local/lib/python2.7/site-packages/IPython/extensions']

In [5]:
file_name = "test_audio1.mp3"

In [6]:
AudioSegment.from_mp3(file_name)

CouldntEncodeError: Encoding failed. ffmpeg/avlib returned error code: 1

Command:['ffmpeg', '-y', '-f', 'wav', '-i', '/tmp/tmpWpupYU', '-f', 'mp3', '/tmp/tmpwHtXT_']

Output from ffmpeg/avlib:

ffmpeg version 3.2.4 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/kian/anaconda3 --disable-doc --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC -I/home/kian/anaconda3/include' --extra-cxxflags='=-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-L/home/kian/anaconda3/lib -lz' --enable-pic --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libx264
  libavutil      55. 34.101 / 55. 34.101
  libavcodec     57. 64.101 / 57. 64.101
  libavformat    57. 56.101 / 57. 56.101
  libavdevice    57.  1.100 / 57.  1.100
  libavfilter     6. 65.100 /  6. 65.100
  libavresample   3.  1.  0 /  3.  1.  0
  libswscale      4.  2.100 /  4.

In [10]:
g = gTTS("Hello")

In [11]:
g

In [19]:
f = TemporaryFile()
g.write_to_fp(f)

In [18]:
pydub.AudioSegment

AttributeError: '_io.BufferedRandom' object has no attribute 'size'